# Fine-tuning

In [1]:
from transformers import AutoConfig, AutoTokenizer, TrainingArguments, AutoModel, AutoModelForSequenceClassification, Trainer, pipeline, DataCollatorForTokenClassification, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric, Dataset
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
import torch
import torchinfo


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/user/rajkumardheivanayahi/oauth_callback')}
  warn(msg)
/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('servers!server=rajkumardheivanayahi/", "access'), PosixPath('["access'), PosixPath('servers!user=rajkumardheivanayahi"]')}
  warn(msg)
/data/rajkumardheivanayahi/.local/lib/python3.9/si

In [2]:
checkpoint = 'bert-base-cased'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
tokenizer("Hello World")

{'input_ids': [101, 8667, 1291, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [5]:
tokenizer.tokenize("Hello world full of Chimponzees")

['Hello', 'world', 'full', 'of', 'Chi', '##mpo', '##nz', '##ees']

In [6]:
ids = tokenizer.encode('Hello world')

In [7]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'Hello', 'world', '[SEP]']

In [8]:
df = pd.read_csv('./DATA.csv')

In [9]:
df

,Fwd Packets/s,Bwd Packets/s,Flow Bytes/s,packet,source_port,destination_port,IP len,payload len,IP ttl,IP tos,...,byte_492,byte_493,byte_494,byte_495,byte_496,byte_497,byte_498,byte_499,attack_cat,packet_dat
0,1170,1625,3406409,-1,80,50580,1500,1460,118,0,...,133,218,248,64,68,236,51,110,Normal,1170 1625 3406409 -1 80 50580 1500 1460 118 0 ...
1,1357,1905,3946827,-1,80,50580,1500,1460,118,0,...,133,218,248,64,68,236,51,110,Normal,1357 1905 3946827 -1 80 50580 1500 1460 118 0 ...
2,1223,1747,3661835,-1,80,50580,1500,1460,118,0,...,133,218,248,64,68,236,51,110,Normal,1223 1747 3661835 -1 80 50580 1500 1460 118 0 ...
3,1183,1610,3465091,-1,80,50580,1500,1460,118,0,...,133,218,248,64,68,236,51,110,Normal,1183 1610 3465091 -1 80 50580 1500 1460 118 0 ...
4,464,655,1356121,-1,80,50580,1500,1460,118,0,...,133,218,248,64,68,236,51,110,Normal,464 655 1356121 -1 80 50580 1500 1460 118 0 5 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700736,13,12,7028,-1,20503,33532,313,273,254,0,...,-1,-1,-1,-1,-1,-1,-1,-1,Backdoor,13 12 7028 -1 20503 33532 313 273 254 0 5 0 -1...
1700737,24,19,2579,-1,1014,60251,148,108,253,0,...,-1,-1,-1,-1,-1,-1,-1,-1,Backdoor,24 19 2579 -1 1014 60251 148 108 253 0 5 0 -1 ...
1700738,24,19,2579,-1,1014,60251,148,108,252,0,...,-1,-1,-1,-1,-1,-1,-1,-1,Backdoor,24 19 2579 -1 1014 60251 148 108 252 0 5 0 -1 ...
1700739,24,19,2579,-1,60251,1014,54,14,255,0,...,-1,-1,-1,-1,-1,-1,-1,-1,Backdoor,24 19 2579 -1 60251 1014 54 14 255 0 5 0 -1 67...


In [10]:
df = df[['packet_dat', 'attack_cat']]

In [11]:
df = df.drop_duplicates(subset=['packet_dat'], ignore_index=True)

In [12]:
df

,packet_dat,attack_cat
0,1170 1625 3406409 -1 80 50580 1500 1460 118 0 ...,Normal
1,1357 1905 3946827 -1 80 50580 1500 1460 118 0 ...,Normal
2,1223 1747 3661835 -1 80 50580 1500 1460 118 0 ...,Normal
3,1183 1610 3465091 -1 80 50580 1500 1460 118 0 ...,Normal
4,464 655 1356121 -1 80 50580 1500 1460 118 0 5 ...,Normal
...,...,...
1696826,13 12 7028 -1 20503 33532 313 273 254 0 5 0 -1...,Backdoor
1696827,24 19 2579 -1 1014 60251 148 108 253 0 5 0 -1 ...,Backdoor
1696828,24 19 2579 -1 1014 60251 148 108 252 0 5 0 -1 ...,Backdoor
1696829,24 19 2579 -1 60251 1014 54 14 255 0 5 0 -1 67...,Backdoor


In [13]:
df.attack_cat.value_counts()

DoS Hulk                      596942
Normal                        259650
DDoS                          182866
Heartbleed                    167360
SSH Patator                    85397
DoS                            70239
Exploits                       65551
Generic                        49120
FTP Patator                    45189
Fuzzers                        36198
Reconnaissance                 34645
DoS GoldenEye                  33486
Web Attack - Brute Force       18630
Infiltration                   10774
Worms                           9466
DoS SlowHTTPTest                7999
DoS Slowloris                   6276
Web Attack - XSS                5154
Bot                             5139
Analysis                        2612
Backdoor                        1752
Shellcode                       1498
Port Scan                        843
Web Attack - SQL Injection        45
Name: attack_cat, dtype: int64

In [14]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [15]:
train_df.attack_cat.value_counts()

DoS Hulk                      417967
Normal                        181742
DDoS                          127975
Heartbleed                    117056
SSH Patator                    59905
DoS                            49047
Exploits                       45931
Generic                        34343
FTP Patator                    31684
Fuzzers                        25409
Reconnaissance                 24250
DoS GoldenEye                  23355
Web Attack - Brute Force       13014
Infiltration                    7514
Worms                           6552
DoS SlowHTTPTest                5567
DoS Slowloris                   4424
Bot                             3679
Web Attack - XSS                3605
Analysis                        1819
Backdoor                        1245
Shellcode                       1063
Port Scan                        605
Web Attack - SQL Injection        30
Name: attack_cat, dtype: int64

In [16]:
test_df.attack_cat.value_counts()

DoS Hulk                      178975
Normal                         77908
DDoS                           54891
Heartbleed                     50304
SSH Patator                    25492
DoS                            21192
Exploits                       19620
Generic                        14777
FTP Patator                    13505
Fuzzers                        10789
Reconnaissance                 10395
DoS GoldenEye                  10131
Web Attack - Brute Force        5616
Infiltration                    3260
Worms                           2914
DoS SlowHTTPTest                2432
DoS Slowloris                   1852
Web Attack - XSS                1549
Bot                             1460
Analysis                         793
Backdoor                         507
Shellcode                        435
Port Scan                        238
Web Attack - SQL Injection        15
Name: attack_cat, dtype: int64

In [17]:
train_df.to_csv('./DATA_TRAIN.csv', index=False)

In [18]:
test_df.to_csv('./DATA_TEST.csv', index=False)

In [19]:
train_df = train_df.groupby('attack_cat').apply(lambda x: x.sample(min(len(x), 100000))).reset_index(drop=True)

In [20]:
train_df.attack_cat.value_counts()

DDoS                          100000
DoS Hulk                      100000
Normal                        100000
Heartbleed                    100000
SSH Patator                    59905
DoS                            49047
Exploits                       45931
Generic                        34343
FTP Patator                    31684
Fuzzers                        25409
Reconnaissance                 24250
DoS GoldenEye                  23355
Web Attack - Brute Force       13014
Infiltration                    7514
Worms                           6552
DoS SlowHTTPTest                5567
DoS Slowloris                   4424
Bot                             3679
Web Attack - XSS                3605
Analysis                        1819
Backdoor                        1245
Shellcode                       1063
Port Scan                        605
Web Attack - SQL Injection        30
Name: attack_cat, dtype: int64

In [21]:
ros = RandomOverSampler(random_state=42)

In [22]:
X, y = ros.fit_resample(train_df[['packet_dat']], train_df[['attack_cat']])

In [23]:
data = pd.concat([X,y], axis=1)

In [24]:
data

,packet_dat,attack_cat
0,14 11 2798 -1 62088 80 229 189 62 0 5 3 -1 71 ...,Analysis
1,24 19 4987 -1 80 63691 473 433 253 0 5 0 -1 72...,Analysis
2,8 8 1637 -1 63648 80 230 190 63 0 5 3 -1 71 69...,Analysis
3,30 24 6404 -1 80 65401 512 472 253 0 5 0 -1 72...,Analysis
4,21 17 4740 -1 13591 80 229 189 63 0 5 3 -1 71 ...,Analysis
...,...,...
2399995,12 70 91946 -1 80 33820 1500 1460 253 0 5 0 -1...,Worms
2399996,14 77 103183 -1 80 49608 1500 1460 252 0 5 0 -...,Worms
2399997,14 87 119898 -1 80 63800 1500 1460 252 0 5 0 -...,Worms
2399998,18 108 141828 -1 80 54215 1500 1460 253 0 5 0 ...,Worms


In [25]:
data.attack_cat.value_counts()

Analysis                      100000
Backdoor                      100000
Web Attack - XSS              100000
Web Attack - SQL Injection    100000
Web Attack - Brute Force      100000
Shellcode                     100000
SSH Patator                   100000
Reconnaissance                100000
Port Scan                     100000
Normal                        100000
Infiltration                  100000
Heartbleed                    100000
Generic                       100000
Fuzzers                       100000
FTP Patator                   100000
Exploits                      100000
DoS Slowloris                 100000
DoS SlowHTTPTest              100000
DoS Hulk                      100000
DoS GoldenEye                 100000
DoS                           100000
DDoS                          100000
Bot                           100000
Worms                         100000
Name: attack_cat, dtype: int64

In [26]:
classes = data['attack_cat'].unique()

target_map = {class_name: index for index, class_name in enumerate(sorted(classes))}

In [27]:
target_map

{'Analysis': 0,
 'Backdoor': 1,
 'Bot': 2,
 'DDoS': 3,
 'DoS': 4,
 'DoS GoldenEye': 5,
 'DoS Hulk': 6,
 'DoS SlowHTTPTest': 7,
 'DoS Slowloris': 8,
 'Exploits': 9,
 'FTP Patator': 10,
 'Fuzzers': 11,
 'Generic': 12,
 'Heartbleed': 13,
 'Infiltration': 14,
 'Normal': 15,
 'Port Scan': 16,
 'Reconnaissance': 17,
 'SSH Patator': 18,
 'Shellcode': 19,
 'Web Attack - Brute Force': 20,
 'Web Attack - SQL Injection': 21,
 'Web Attack - XSS': 22,
 'Worms': 23}

In [28]:
data['target'] = data['attack_cat'].map(target_map)

In [29]:
data

,packet_dat,attack_cat,target
0,14 11 2798 -1 62088 80 229 189 62 0 5 3 -1 71 ...,Analysis,0
1,24 19 4987 -1 80 63691 473 433 253 0 5 0 -1 72...,Analysis,0
2,8 8 1637 -1 63648 80 230 190 63 0 5 3 -1 71 69...,Analysis,0
3,30 24 6404 -1 80 65401 512 472 253 0 5 0 -1 72...,Analysis,0
4,21 17 4740 -1 13591 80 229 189 63 0 5 3 -1 71 ...,Analysis,0
...,...,...,...
2399995,12 70 91946 -1 80 33820 1500 1460 253 0 5 0 -1...,Worms,23
2399996,14 77 103183 -1 80 49608 1500 1460 252 0 5 0 -...,Worms,23
2399997,14 87 119898 -1 80 63800 1500 1460 252 0 5 0 -...,Worms,23
2399998,18 108 141828 -1 80 54215 1500 1460 253 0 5 0 ...,Worms,23


In [30]:
data = data[['packet_dat', 'target']]

In [31]:
data.columns = ['packet', 'label']

In [32]:
data

,packet,label
0,14 11 2798 -1 62088 80 229 189 62 0 5 3 -1 71 ...,0
1,24 19 4987 -1 80 63691 473 433 253 0 5 0 -1 72...,0
2,8 8 1637 -1 63648 80 230 190 63 0 5 3 -1 71 69...,0
3,30 24 6404 -1 80 65401 512 472 253 0 5 0 -1 72...,0
4,21 17 4740 -1 13591 80 229 189 63 0 5 3 -1 71 ...,0
...,...,...
2399995,12 70 91946 -1 80 33820 1500 1460 253 0 5 0 -1...,23
2399996,14 77 103183 -1 80 49608 1500 1460 252 0 5 0 -...,23
2399997,14 87 119898 -1 80 63800 1500 1460 252 0 5 0 -...,23
2399998,18 108 141828 -1 80 54215 1500 1460 253 0 5 0 ...,23


In [33]:
raw_dataset = Dataset.from_pandas(data)

In [34]:
raw_dataset

Dataset({
    features: ['packet', 'label'],
    num_rows: 2400000
})

In [35]:
raw_dataset = raw_dataset.train_test_split(test_size=0.3, seed=42)

In [36]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['packet', 'label'],
        num_rows: 1680000
    })
    test: Dataset({
        features: ['packet', 'label'],
        num_rows: 720000
    })
})

In [37]:
raw_dataset['train'][0]

{'packet': '12 74 103446 -1 80 32283 1500 1460 253 0 5 0 -1 104 154 241 129 0 196 192 141 87 156 55 79 75 21 37 12 221 156 195 186 244 109 63 40 142 167 137 151 4 92 61 62 55 210 40 32 8 150 219 248 14 72 227 14 54 141 178 62 207 137 118 70 225 202 192 232 232 85 51 121 107 233 228 144 69 246 166 219 114 105 25 233 190 9 244 236 88 203 97 81 99 237 31 78 188 128 1 188 173 162 190 213 53 54 178 255 31 251 119 148 7 109 195 242 128 58 142 235 183 162 228 122 187 244 233 143 9 16 237 235 149 96 197 12 81 191 166 243 76 243 52 0 240 180 116 47 62 186 84 177 177 149 41 121 228 16 107 87 76 54 41 162 180 222 19 148 92 167 94 247 254 242 210 189 171 54 171 199 121 143 121 122 175 235 115 10 160 237 227 141 118 10 203 192 102 128 195 188 151 246 180 136 114 19 241 8 59 100 89 39 23 109 226 149 95 26 235 86 192 0 3 142 35 32 171 193 227 249 179 22 210 185 132 104 56 4 18 58 128 12 222 144 28 235 129 3 169 44 30 74 152 113 157 124 253 206 197 210 18 63 58 120 181 181 154 245 102 161 226 255 136 

In [38]:
def tokenize_batch(batch):
  return tokenizer(batch['packet'], truncation=True)

In [39]:
tokenized_datasets = raw_dataset.map(function=tokenize_batch, batched=True)

Map:   0%|          | 0/1680000 [00:00<?, ? examples/s]

Map:   0%|          | 0/720000 [00:00<?, ? examples/s]

In [40]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['packet', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1680000
    })
    test: Dataset({
        features: ['packet', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 720000
    })
})

In [41]:
config = AutoConfig.from_pretrained(checkpoint)

In [42]:
config

BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [43]:
config.label2id = target_map

In [44]:
config.id2label = {v:k for k,v in target_map.items()}

In [45]:
config

BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Analysis",
    "1": "Backdoor",
    "2": "Bot",
    "3": "DDoS",
    "4": "DoS",
    "5": "DoS GoldenEye",
    "6": "DoS Hulk",
    "7": "DoS SlowHTTPTest",
    "8": "DoS Slowloris",
    "9": "Exploits",
    "10": "FTP Patator",
    "11": "Fuzzers",
    "12": "Generic",
    "13": "Heartbleed",
    "14": "Infiltration",
    "15": "Normal",
    "16": "Port Scan",
    "17": "Reconnaissance",
    "18": "SSH Patator",
    "19": "Shellcode",
    "20": "Web Attack - Brute Force",
    "21": "Web Attack - SQL Injection",
    "22": "Web Attack - XSS",
    "23": "Worms"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Analysis": 0,
    "Backdoor": 1,
    "Bot": 2

In [46]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [47]:
torchinfo.summary(model)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              22,268,928
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           18,456
Total params: 1

In [48]:
training_args = TrainingArguments(
    output_dir='BERT',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128
    )

/data/rajkumardheivanayahi/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [49]:
def compute_metric(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average='macro')
  return {'accuracy':acc, 'f1-score':f1}

In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    compute_metrics = compute_metric
    )

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [45]:
trainer

In [46]:
trainer.state.log_history

[{'loss': 0.7694,
  'learning_rate': 4.966644429619747e-05,
  'epoch': 0.03,
  'step': 500},
 {'loss': 0.1494,
  'learning_rate': 4.933288859239493e-05,
  'epoch': 0.07,
  'step': 1000},
 {'loss': 0.1019,
  'learning_rate': 4.89993328885924e-05,
  'epoch': 0.1,
  'step': 1500},
 {'loss': 0.0828,
  'learning_rate': 4.866577718478986e-05,
  'epoch': 0.13,
  'step': 2000},
 {'loss': 0.0724,
  'learning_rate': 4.833222148098733e-05,
  'epoch': 0.17,
  'step': 2500},
 {'loss': 0.0656,
  'learning_rate': 4.799866577718479e-05,
  'epoch': 0.2,
  'step': 3000},
 {'loss': 0.0598,
  'learning_rate': 4.766511007338226e-05,
  'epoch': 0.23,
  'step': 3500},
 {'loss': 0.0585,
  'learning_rate': 4.733155436957972e-05,
  'epoch': 0.27,
  'step': 4000},
 {'loss': 0.0563,
  'learning_rate': 4.699799866577719e-05,
  'epoch': 0.3,
  'step': 4500},
 {'loss': 0.0513,
  'learning_rate': 4.6664442961974656e-05,
  'epoch': 0.33,
  'step': 5000},
 {'loss': 0.0488,
  'learning_rate': 4.633088725817212e-05,
  'e

# Testing

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('./DATA_TEST.csv')

In [8]:
df

,packet_dat,attack_cat
0,75 75 864 -1 80 48846 2948 2896 64 0 8 0 -1 32...,DoS Hulk
1,0 0 2 -1 80 56471 10175 10135 64 0 5 3 -1 112 ...,DDoS
2,500000 0 0 -1 80 46678 1500 1448 64 0 8 0 -1 7...,DoS Hulk
3,240 481 956354 -1 80 33554 1500 1448 64 0 8 0 ...,DoS Hulk
4,111 16 147912 -1 38156 25 1500 1460 254 0 5 0 ...,DoS
...,...,...
509045,30 40 116520 -1 80 56105 5795 5755 64 0 5 2 -1...,DDoS
509046,666666 0 0 -1 54614 80 422 370 62 0 8 3 -1 71 ...,DoS Hulk
509047,500000 0 0 -1 41110 80 428 376 62 0 8 3 -1 71 ...,DoS Hulk
509048,77 12 103767 -1 49888 25 1500 1460 254 0 5 0 -...,Generic


In [36]:
df.attack_cat.value_counts()

DoS Hulk                      178975
Normal                         77908
DDoS                           54891
Heartbleed                     50304
SSH Patator                    25492
DoS                            21192
Exploits                       19620
Generic                        14777
FTP Patator                    13505
Fuzzers                        10789
Reconnaissance                 10395
DoS GoldenEye                  10131
Web Attack - Brute Force        5616
Infiltration                    3260
Worms                           2914
DoS SlowHTTPTest                2432
DoS Slowloris                   1852
Web Attack - XSS                1549
Bot                             1460
Analysis                         793
Backdoor                         507
Shellcode                        435
Port Scan                        238
Web Attack - SQL Injection        15
Name: attack_cat, dtype: int64

In [10]:
from transformers import pipeline

In [11]:
pipe = pipeline("text-classification", model="mwz/NID_tr", device=0)

In [19]:
predictions = pipe(df['packet_dat'].to_list(), truncation=True)

In [29]:
pred = [pred['label'] for pred in predictions]

In [21]:
from sklearn.metrics import accuracy_score, f1_score

In [31]:
accuracy_score(df['attack_cat'], pred)

0.9948453000687555

In [34]:
f1_score(df['attack_cat'], pred, average='micro')

0.9948453000687555

In [35]:
f1_score(df['attack_cat'], pred, average='macro')

0.9846018986149065